In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [3]:
#Read the excel file and store the sheets into dataframes.
df = pd.ExcelFile('C:/Users/jaina/OneDrive/Desktop/Python Hackathon 2/V02Max_Combined.xlsx')
kay = pd.read_excel(df, sheet_name = 'Kayaking')
ath = pd.read_excel(df, sheet_name = 'Athletes')
tri = pd.read_excel(df, sheet_name = 'Triathlon')
fen = pd.read_excel(df, sheet_name = 'Fencing')

Question 1) Join all sport-related tables to produce one dataframe that has all athletes values. The final table should only have the same 5 columns: ID, time,power,RR, vo2, power. Write this dataframe to an excel file

In [40]:
# Assuming you have the dataframes kay, ath, tri, fen defined

# Check for missing columns (optional)
print(kay.columns)
print(ath.columns)
print(tri.columns)
print(fen.columns)

# Add placeholder column if necessary
if 'VO2' not in kay.columns:
    kay['VO2'] = np.nan

# Concatenate the dataframes
combined_df = pd.concat([kay, ath, tri, fen], ignore_index=True)

# Select the desired columns
final_df = combined_df[['ID', 'time', 'power', 'RR', 'VO2']]

# Write to Excel
writer = pd.ExcelWriter('combined_data.xlsx')
final_df.to_excel(writer, index=False)
writer.close()

Index(['ID', 'time', 'RR', 'VO2', 'power', 'vo2'], dtype='object')
Index(['ID', 'age', 'Weight', 'Height', 'P_vt1', 'P_vt2', 'sport', 'Gender',
       'Ethnicity', 'First Name', 'Last Name'],
      dtype='object')
Index(['ID', 'time', 'RR', 'VO2', 'power'], dtype='object')
Index(['ID', 'time', 'RR', 'VO2', 'power'], dtype='object')


##Question 2 Divide the rows into 3 parts: Warm up, GET(graded exercise test) and cool down using the data def. Label each row accordingly

In [ ]:
#df = pd.ExcelFile('C:/Users/jaina/OneDrive/Desktop/Python Hackathon 2/combined_data.xlsx')# 
df = pd.DataFrame(data)

# Define thresholds for warm-up and cool-down (adjust as needed)
warm_up_threshold = 15
cool_down_threshold = 15

# Function to determine the phase based on value
def determine_phase(value):
    if value < warm_up_threshold:
        return 'Warm-up'
    elif value > cool_down_threshold:
        return 'GET'
    else:
        return 'Cool-down'

# Apply the function to create a new column 'phase'
df['phase'] = df['value'].apply(determine_phase)

print(df)

Question 3 What was the average time each tested lasted excluding warm up for all athletes?